# Web Scrapper

- Obtener resultados de google
- Obtener los párrafos de cada resultado
- Preprocesar y guardar

In [42]:
from util.scrapper import *
from models.lda_utils import get_keywords
import re
is_well_formed_link = re.compile(r'^https?://.+/.+$')

def is_useful(string):
    return string != '\n'

keywords = get_keywords()
data = []
for keyword in keywords:
    try:
        results = scrape_google(keyword, 5, "es")
        for result in results:
            data.append((keyword, result))
    except Exception as e:
        print(e)
    finally:
        time.sleep(10)


links = [(keyword, result['link']) for keyword, result in data if is_well_formed_link.match(result['link'])]

In [43]:
get_keywords()

['estrategia precio red coste control caso usuario copia positivo preguntar',
 'experiencia cliente masivo valor sacrificio oferta individuo necesidad campo tema',
 'cliente experiencia valor masivo negocio necesidad diferencia ciberespacio misma precio',
 'cliente marca social comunidad conectividad horizontal mundo precio red concepto',
 'emergente conducta organismo comportamiento complejo agente desarrollar superior regla complejas',
 'plataforma conocimiento red fuente dato colaborativa valor com modelo humano',
 'adoptar innovador individuo consumidor modelo gente pc grupo rol coeficiente',
 'rifkin industrial infraestructura marginal actual ley decir sociedad factura capitalista',
 'coste tarea global compartir concepto favorecer ley coase emergencia capacidad',
 'largo cola ofrecer venta regla cliente demanda nicho oferta minorista']

In [44]:
links

[('estrategia precio red coste control caso usuario copia positivo preguntar',
  'https://br.escueladenegociosydireccion.com/business/marketing-ventas/estrategias-de-precios-en-un-mercado-competitivo/'),
 ('estrategia precio red coste control caso usuario copia positivo preguntar',
  'https://es.semrush.com/blog/estrategia-de-precios/'),
 ('estrategia precio red coste control caso usuario copia positivo preguntar',
  'http://diposit.ub.edu/dspace/bitstream/2445/66261/1/TFM_MOI_Pedrero-Yolanda-jun2015.pdf'),
 ('estrategia precio red coste control caso usuario copia positivo preguntar',
  'https://www.tesisenred.net/bitstream/handle/10803/385276/tesdoc_a2016_roman_david_analisis_promociones.pdf?sequence=1&isAllowed=y'),
 ('estrategia precio red coste control caso usuario copia positivo preguntar',
  'https://e-archivo.uc3m.es/bitstream/handle/10016/22498/rosario_rivera_tesis.pdf'),
 ('experiencia cliente masivo valor sacrificio oferta individuo necesidad campo tema',
  'http://tdx.cat/bi

In [45]:
paragraphs = []

In [46]:
import urllib.request
from bs4 import BeautifulSoup

def is_useful(p):
    return len(p) > 3

for keyword, link in links:
    try:
        content = urllib.request.urlopen(link)
        read_content = content.read()
        soup = BeautifulSoup(read_content,'html.parser')
        pAll = soup.find_all('p')
    except:
        pass
    paragraphs.append((keyword, link, list(filter(is_useful, [p.text for p in pAll]))))

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.
Some characters could not be decoded, and were replaced with REPLACEMENT CHA

In [47]:
for ix, (keyword, link, p) in enumerate(paragraphs):
    paragraphs[ix] = keyword, link, list(filter(is_useful, p))

In [48]:
from document import Document
preprocess_list = ['lemmatized_string', 'sentences', 'preprocessed_sentences', 'paragraphs', 'preprocessed_paragraphs']
documents = []

for keywords, link, ps in paragraphs:
    doc = Document()
    doc.from_paragraphs(paragraphs = ps, preprocess_list=preprocess_list)
    doc.title = link
    doc.topic = keywords
    documents.append(doc)


In [49]:
def process_file(document):
    return [document.title, document.title, document.string, document.word_count(), document.type_count(), document.sentences,\
                       document.lemmatized_string, document.stemmed_string, document.simple_preprocessed_string, document.topic, \
                       document.named_entities, document.bigrams, document.trigrams, document.lemmatized_bigrams, \
                       document.lemmatized_trigrams, document.stemmed_bigrams, document.stemmed_trigrams, \
                       document.simple_preprocessed_bigrams, document.simple_preprocessed_trigrams, document.preprocessed_sentences,
                       document.paragraphs, document.preprocessed_paragraphs, document.urls]

scrapped = [process_file(doc) for doc in documents]

In [50]:
import pandas as pd
dataframe_scrapped = pd.DataFrame(data = scrapped, columns=['original_path', 'document_title', 'string', 'word_count', 'type_count', 'sentences','lemmatized_text', 'stemmed_text', 'simple_preprocessed', 'topic', 'named_entities', 'tokens_bigrams', 'tokens_trigrams', 'lemmatized_bigrams', 'lemmatized_trigrams', 'stemmed_bigrams', 'stemmed_trigrams', 'simple_preprocessed_bigrams', 'simple_preprocessed_trigrams', 'preprocessed_sentences', 'paragraphs', 'preprocessed_paragraphs', 'urls'])

In [51]:
dataframe_scrapped.to_csv('../data/scrapped.csv')

In [52]:
dataframe_scrapped

,original_path,document_title,string,word_count,type_count,sentences,lemmatized_text,stemmed_text,simple_preprocessed,topic,...,lemmatized_bigrams,lemmatized_trigrams,stemmed_bigrams,stemmed_trigrams,simple_preprocessed_bigrams,simple_preprocessed_trigrams,preprocessed_sentences,paragraphs,preprocessed_paragraphs,urls
0,https://br.escueladenegociosydireccion.com/bus...,https://br.escueladenegociosydireccion.com/bus...,Cuando necesitas comprar un producto o contrat...,1605,607,[Cuando necesitas comprar un producto o contra...,"[necesitas, comprar, producto, contratar, serv...",None,None,estrategia precio red coste control caso usuar...,...,None,None,None,None,None,None,None,[Cuando necesitas comprar un producto o contra...,[necesitas comprar producto contratar servicio...,[]
1,https://es.semrush.com/blog/estrategia-de-prec...,https://es.semrush.com/blog/estrategia-de-prec...,Cuando necesitas comprar un producto o contrat...,1605,607,[Cuando necesitas comprar un producto o contra...,"[necesitas, comprar, producto, contratar, serv...",None,None,estrategia precio red coste control caso usuar...,...,None,None,None,None,None,None,None,[Cuando necesitas comprar un producto o contra...,[necesitas comprar producto contratar servicio...,[]
2,http://diposit.ub.edu/dspace/bitstream/2445/66...,http://diposit.ub.edu/dspace/bitstream/2445/66...,,0,0,[],[],None,None,estrategia precio red coste control caso usuar...,...,None,None,None,None,None,None,None,[],[],[]
3,https://www.tesisenred.net/bitstream/handle/10...,https://www.tesisenred.net/bitstream/handle/10...,,0,0,[],[],None,None,estrategia precio red coste control caso usuar...,...,None,None,None,None,None,None,None,[],[],[]
4,https://e-archivo.uc3m.es/bitstream/handle/100...,https://e-archivo.uc3m.es/bitstream/handle/100...,,0,0,[],[],None,None,estrategia precio red coste control caso usuar...,...,None,None,None,None,None,None,None,[],[],[]
5,http://tdx.cat/bitstream/handle/10803/396345/T...,http://tdx.cat/bitstream/handle/10803/396345/T...,,0,0,[],[],None,None,experiencia cliente masivo valor sacrificio of...,...,None,None,None,None,None,None,None,[],[],[]
6,https://www.tesisenred.net/bitstream/handle/10...,https://www.tesisenred.net/bitstream/handle/10...,,0,0,[],[],None,None,experiencia cliente masivo valor sacrificio of...,...,None,None,None,None,None,None,None,[],[],[]
7,https://www.tesisenred.net/bitstream/handle/10...,https://www.tesisenred.net/bitstream/handle/10...,,0,0,[],[],None,None,experiencia cliente masivo valor sacrificio of...,...,None,None,None,None,None,None,None,[],[],[]
8,https://journals.openedition.org/polis/4887,https://journals.openedition.org/polis/4887,Revista Latinoamericana InicioNúmeros15Lente ...,11741,2996,[ Revista Latinoamericana InicioNúmeros15Lente...,"[revista, latinoamericano, debate, buscar, pre...",None,None,experiencia cliente masivo valor sacrificio of...,...,None,None,None,None,None,None,None,"[ Revista Latinoamericana , InicioNúmeros15Len...","[revista latinoamericano, debate, buscar, pres...",[]
9,https://www.navarra.es/NR/rdonlyres/B31A650F-3...,https://www.navarra.es/NR/rdonlyres/B31A650F-3...,,0,0,[],[],None,None,experiencia cliente masivo valor sacrificio of...,...,None,None,None,None,None,None,None,[],[],[]
